# 05/12/2023

The purpose of this notebook is to test the SMART_CLIP model

In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
from pathlib import Path
import torch
from datasets.dataset_utils import pad_and_mask, tokenise_and_mask_encoder
from datasets.generic_index_dataset import GenericIndexedModule
from models.chemformer.tokeniser import MolEncTokeniser
from models.chemformer.utils import REGEX

from models.smart_clip import SMART_CLIP

path = "./tempdata/epoch=76-step=264264.ckpt"
vocab_path = "tempdata/chemformer/bart_vocab.txt"
chem_token_start = 272

In [7]:
tokeniser = MolEncTokeniser.from_vocab_file(
  vocab_path, REGEX, chem_token_start
)
direct = Path("tempdata/SMILES_dataset")
features = ["HSQC", "SMILES"]
def tame(a):
  return tokenise_and_mask_encoder(a, tokeniser)
feature_handlers = [pad_and_mask, tame]
gim = GenericIndexedModule(direct, features, feature_handlers, len_override = 5)
gim.setup("fit")
train_dl = gim.train_dataloader()

In [21]:
for batch in train_dl:
  obj = batch
  for item in obj:
    for k in item:
      if type(item[k]) is torch.Tensor:
        print(k, item[k].size())
        item[k] = item[k].cuda()
  break

sequence torch.Size([5, 21, 3])
padding_mask torch.Size([5, 21])
encoder_inputs torch.Size([5, 41])
encoder_mask torch.Size([5, 41])


In [25]:
kwargs = {
  "chemformer_path": "tempdata/chemformer/model.ckpt",
  "projection_dim": 256,
  "lr": 1e-5,
  "coord_enc": "sce",
  "enc_args": {
    "wavelength_bounds": ((0.01, 150), (0.01, 150))
  },
  "dim_model": 128
}
model = SMART_CLIP(**kwargs).cuda()

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/migration.py:201: PossibleUserWarning: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
  rank_zero_warn(
Lightning automatically upgraded your loaded checkpoint from v1.2.3 to v2.0.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file tempdata/chemformer/model.ckpt`
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:157: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['decoder.layers.0.self_attn.in_proj_weight', 'decoder.layers.0.self_attn.in_proj_bias', 'decoder.layers.0.self_attn.out_proj.weight',

In [30]:
hsqc, smiles = model(obj)

hsqc_out.size()=torch.Size([5, 128])
chemformer_out.size()=torch.Size([5, 512])


tensor([[True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True]], device='cuda:0')